In [4]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch 

In [5]:
root_path = Path('./data/')
n_optim = 50
results = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            print(os.path.join(subdir, file))
            _, ds, query, budget, params, seed,  = subdir.split('/')

            if ds not in results:
                results[ds] = {}

            if query not in results[ds]:
                results[ds][query] = {}

            if budget not in results[ds][query]:
                results[ds][query][budget] = {}

            if params not in results[ds][query][budget]:
                results[ds][query][budget][params] = {}

            if seed not in results[ds][query][budget][params]:
                results[ds][query][budget][params][seed] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results[ds][query][budget][params][seed] = res

data/CIFAR10/random/budget2000/lr0.001_wd0.05/seed1/results.json
data/CIFAR10/random/budget2000/lr0.001_wd0.05/seed3/results.json
data/CIFAR10/random/budget2000/lr0.001_wd0.05/seed2/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.0005/seed1/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.0005/seed3/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.0005/seed2/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.05/seed1/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.05/seed3/results.json
data/CIFAR10/random/budget2000/lr0.01_wd0.05/seed2/results.json
data/CIFAR10/random/budget2000/lr0.1_wd0.005/seed1/results.json
data/CIFAR10/random/budget2000/lr0.1_wd0.005/seed3/results.json
data/CIFAR10/random/budget2000/lr0.1_wd0.005/seed2/results.json
data/CIFAR10/random/budget4000/lr0.001_wd0.05/seed1/results.json
data/CIFAR10/random/budget4000/lr0.001_wd0.05/seed3/results.json
data/CIFAR10/random/budget4000/lr0.001_wd0.05/seed2/results.json
data/CIFAR10/random/budget40

In [6]:
test_accuracys = {}
for ds in results:
    test_accuracys[ds] = {}
    for query in results[ds]:
        test_accuracys[ds][query] = {}
        for budget in results[ds][query]:
            test_accuracys[ds][query][budget] = {}
            for params in results[ds][query][budget]:
                test_accuracys[ds][query][budget][params] = []
                for seed in results[ds][query][budget][params]:
                    # Extract final test acc from json file
                    final_test_acc = results[ds][query][budget][params][seed]['test_stats']['test_acc']
                    test_accuracys[ds][query][budget][params].append(final_test_acc*100)

In [7]:
# Validate that each param group has 3 finished runs
unfinished_runs = []
for ds in test_accuracys:
    for query in test_accuracys[ds]:
        for budget in test_accuracys[ds][query]:
            for params in test_accuracys[ds][query][budget]:
                if len(test_accuracys[ds][query][budget][params]) != 3:
                    print("Assertion: The run on ds "+str(ds)+" with query strategy "+str(query)+ " and parameters "+params+" and budget "+budget+" has not had 3 finished runs yet and \n will be left out for the following result table!")
                    unfinished_runs.append([ds, query, budget, params])
                
# Remove unfinished runs
for ds, query, budget, params in unfinished_runs:
    test_accuracys[ds][query][budget].pop(params)

In [8]:
for ds in test_accuracys:
    df = {}
    for query in ['coreset']:
        for budget in test_accuracys[ds][query]:
            key = query+budget[-4:]
            df[key] = {}
            for params in test_accuracys[ds][query][budget]:
                df[key][params] = str(np.mean(test_accuracys[ds][query][budget][params]).round(2)) + ' +/- ' + str(np.std(test_accuracys[ds][query][budget][params]).round(2))
    df = pd.DataFrame(df)
    #df.to_csv('./result_tables/results.csv')
    print('#################### '+str(ds)+' ###################')
    print(df)

#################### CIFAR10 ###################
                    coreset2000     coreset4000
lr0.001_wd0.05    78.42 +/- 0.5  85.26 +/- 0.54
lr0.01_wd0.0005   75.57 +/- 0.4  83.93 +/- 1.41
lr0.01_wd0.05    77.16 +/- 0.79   83.7 +/- 0.43
lr0.1_wd0.005     76.5 +/- 0.63   84.39 +/- 0.9


In [19]:
max_key = np.argmax( 
    [np.mean( 
        [results['CIFAR10']['coreset']['budget4000']['lr0.01_wd0.05']['seed3']['ray-results'][j+i]['res']['val_acc'] for i in range(3)]
    ) 
        for j in range(0,150,3)]
)*3
print(results['CIFAR10']['coreset']['budget4000']['lr0.01_wd0.0005']['seed1']['ray-results'][max_key]['conf'])

{'lr': 0.03692096069251086, 'weight_decay': 0.0010889483270134419, '__trial_index__': 0}
